In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# it smells like cheetos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# In a Colab cell
%pip install -q "pandas==2.2.2" "requests==2.32.4" \
  "opentelemetry-api==1.37.0" "opentelemetry-sdk==1.37.0" \
  "opentelemetry-proto==1.37.0" "opentelemetry-exporter-otlp-proto-common==1.37.0" \
  "opentelemetry-exporter-otlp-proto-http==1.37.0"

# Optional: verify no broken reqs
!pip check

# Then: Runtime → Restart runtime (important)



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-grpc 1.38.0 requires opentelemetry-exporter-otlp-proto-common==1.38.0, but you have opentelemetry-exporter-otlp-proto-common 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.38.0 requires opentelemetry-proto==1.38.0, but you have opentelemetry-proto 1.37.0 which is incompatible.
opentelemetry-exporter-otlp-proto-grpc 1.38.0 requires opentelemetry-sdk~=1.38.0, but you have opentelemetry-sdk 1.37.0 which is incompatible.
langchain-community 0.4 requires requests<3.0.0,>=2.32.5, but you have requests 2.32.4 which is incompatible.
ipython 7.34.0 requires jedi, which is not installed.
opentelemetry-exporter-otlp-proto-grpc 1.38.0 has requirement opentelemetry-exporter-otlp-proto-common==1.38.0, but you have opentelemetry-exporter-otlp-proto-common 1.37.0.
opentele

# Parse and Chunk sectional contents from CSA and SIP documents

In [ ]:
pip install pymupdf pandas langchain openai chromadb sentence-transformers langchain_community langchain_chroma --upgrade

  Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.38.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_proto-1.38.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.59b0-py3-none-any.whl.metadata (2.4 kB)
Using cached pandas-2.3.3-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.4 MB)
Using cached opentelemetry_exporter_otlp_proto_common-1.38.0-py3-none-any.whl (18 kB)
Using cached opentelemetry_proto-1.38.0-py3-none-any.whl (72 kB)
Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl (132 kB)
Using cached opentelemetry_api-1.38.0-py3-none-any.whl (65 kB)
Using cached opentelemetry_semantic_co

In [ ]:
# @title
import os
import re
import fitz  # PyMuPDF
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter # Langchain update the module name as langchain_text_splitters

# Configuration
PDF_DIRECTORY = "/content/drive/MyDrive/CalWorks/Vector Database/PDFs"  # Set this to your PDF folder path
OUTPUT_DIRECTORY = "/content/drive/MyDrive/CalWorks/Vector Database/Output" # Define output directory

county_names = [
    "Alameda", "Alpine", "Amador", "Butte", "Calaveras", "Colusa", "Contra Costa",
    "Del Norte", "El Dorado", "Fresno", "Glenn", "Humboldt", "Imperial", "Inyo",
    "Kern", "Kings", "Lake", "Lassen", "Los Angeles", "Madera", "Marin", "Mariposa",
    "Mendocino", "Merced", "Modoc", "Mono", "Monterey", "Napa", "Nevada", "Orange",
    "Placer", "Plumas", "Riverside", "Sacramento", "San Benito", "San Bernardino",
    "San Diego", "San Francisco", "San Joaquin", "San Luis Obispo", "San Mateo",
    "Santa Barbara", "Santa Clara", "Santa Cruz", "Shasta", "Sierra", "Siskiyou",
    "Solano", "Sonoma", "Stanislaus", "Sutter", "Tehama", "Trinity", "Tulare",
    "Tuolumne", "Ventura", "Yolo", "Yuba"
]

# Aliases for counties based on common filename patterns
alias_map = {
    "icdss": "Imperial",
    "lacdss": "Los Angeles",
    "ocss": "Orange",
    "scc": "Santa Clara",
    "sbcs": "San Bernardino",
    # Add more aliases as needed
}

def infer_metadata_from_filename(filename):
    # Clean and normalize
    name_clean = filename.lower().replace("_", " ").replace("-", " ").replace(".pdf", "")
    name_compressed = re.sub(r"\s+", "", name_clean)

    # Report type logic
    report_type = "Unknown" # Initialize report_type
    if "sip_pr" in name_clean:
        report_type = "Cal-SIP-PR"
    elif "sip" in name_clean or "system improvement" in name_clean:
        report_type = "Cal-SIP"
    elif (
        "csa" in name_clean
        or "self-assessment" in name_clean
        or "calworks self-assessment" in name_clean
        or "county self-assessment" in name_clean
        or "fatal flaw" in name_clean
    ):
        report_type = "Cal-CSA"


    # Try full county match
    normalized_counties = [c.lower().replace(" ", "") for c in county_names]
    county = "Unknown"
    for orig, compressed in zip(county_names, normalized_counties):
        if compressed in name_compressed:
            county = orig
            break

    # If not found, try alias map
    if county == "Unknown":
        for alias, full_name in alias_map.items():
            if alias in name_compressed:
                county = full_name
                break

    return {
        "file": filename,
        "county": county,
        "report_type": report_type
    }


# A Table of Contents is needed
def extract_sections_via_toc(pdf_path, county, report_type, toc_max_pages=5):
    import fitz
    import re

    doc = fitz.open(pdf_path)
    max_pages = len(doc)
    section_entries = []

    toc_lines = []
    for i in range(min(toc_max_pages, len(doc))):
        text = doc[i].get_text()
        lines = text.split("\n")
        for line in lines:
            line = line.strip()
            # Match: Section headers OR named sections like "Introduction", "Executive Summary"
            match = re.match(
                r"^((Section\s+\d+(\.\d+)?[.:]?\s+.+?)|(?:Introduction|Executive Summary|Demographics))\s+\.{3,}\s+(\d{1,3})$",
                line,
                re.IGNORECASE
            )
            if match:
                title = match.group(1).strip()
                page = int(match.group(4))
                toc_lines.append((title, page))

    # Construct section page ranges
    for i, (title, start_page) in enumerate(toc_lines):
        end_page = toc_lines[i + 1][1] - 1 if i + 1 < len(toc_lines) else max_pages
        section_entries.append({
            "county": county,
            "report_type": report_type,
            "section_header": title,
            "start_page": start_page,
            "end_page": end_page
        })

    # Extract section text
    for section in section_entries:
        start = max(0, section["start_page"] - 1)
        end = min(section["end_page"], max_pages)
        text = "".join(doc[p].get_text() for p in range(start, end))
        section["text"] = text.strip()

    return section_entries


def chunk_sections(sections, chunk_size=1000, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " "]
    )
    chunked = []
    for sec in sections:
        if not sec["text"].strip():
            continue
        splits = splitter.split_text(sec["text"])
        for i, chunk in enumerate(splits):
            chunked.append({
                "county": sec["county"],
                "report_type": sec["report_type"],
                "section": sec["section_header"],
                "page": sec["start_page"],
                "chunk_id": f"{sec['section_header']}_chunk{i}",
                "text": chunk
            })
    return chunked

def main():
    # Create output directory if it doesn't exist
    if not os.path.exists(OUTPUT_DIRECTORY):
        os.makedirs(OUTPUT_DIRECTORY)

    pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.endswith(".pdf")]
    all_sections = []

    for file in pdf_files:
        meta = infer_metadata_from_filename(file)
        if meta["county"] == "Unknown" or meta["report_type"] == "Unknown":
            print(f"Skipping: {file} (missing county or type)")
            continue
        try:
            path = os.path.join(PDF_DIRECTORY, file)
            sections = extract_sections_via_toc(path, meta["county"], meta["report_type"])

            if len(sections) == 0:
                print(f"No TOC sections found for: {file}")
            else:
                print(f"Found {len(sections)} sections in {file}")
                all_sections.extend(sections)

        except Exception as e:
            print(f"Error processing {file}: {e}")

    chunks = chunk_sections(all_sections)
    print(f"\n{len(all_sections)} sections to {len(chunks)} chunks")

    output_path = os.path.join(OUTPUT_DIRECTORY, "chunked_sip_csa_output.xlsx")
    df = pd.DataFrame(chunks)
    df.to_excel(output_path, index=False)
    print(f"Saved to {output_path}")

if __name__ == "__main__":
    main()

Found 12 sections in CSA-Summary-Santa-Clara-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-Sacramento-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-San-Francisco-Fatal-Flaw.pdf
Found 12 sections in CSA-Summary-San-Mateo-Fatal-Flaw.pdf
Found 12 sections in CSA Summary_Fresno_Fatal Flaw.pdf
Found 12 sections in CSA Report_Orange.pdf
Found 12 sections in CSA-Summary-Solano-Report.pdf
Found 11 sections in CSA_Sonoma.pdf
Found 12 sections in CSA_Tulare.pdf
Found 12 sections in CSA_Santa_Cruz.pdf
Found 12 sections in CSA_San_Luis_Obispo.pdf
Found 12 sections in CSA_Contra_Costa.pdf
Found 12 sections in CSA_Yolo.pdf
Found 11 sections in CSA_Ventura.pdf
Found 12 sections in CSA-Summary-Alameda-Fatal-Flaw.pdf
Found 12 sections in CSA_San_Diego.pdf
Found 12 sections in CSA Summary-SantaBarbara.pdf
Found 12 sections in CSA-Summary-Placer.pdf
Found 12 sections in CSA_Mariposa.pdf
Found 12 sections in CSA Riversides.pdf
Found 9 sections in CSA Mendocino.pdf
Found 12 sections in CSA-Tehama.pdf
F

PermissionError: [Errno 1] Operation not permitted: '/content/drive/MyDrive/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx'

# ChromaDb

# Other Free and Efficient Embedding Models

In [ ]:
import os
import re
import shutil
import pandas as pd
import chromadb
from chromadb.config import Settings
from langchain_community.embeddings import HuggingFaceEmbeddings  # Use HuggingFaceEmbeddings
from langchain_core.documents import Document  # Langchain Document storage and retrieval
from langchain_community.vectorstores import Chroma # Use langchain_community.vectorstores

# Configuration
XLSX_PATH   = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chunked_sip_csa_output.xlsx"
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"
COLLECTION  = "sip_csa_chunks"

# Normalize Text
def normalize_text(text: str) -> str:
    text = text.strip()
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"‐|–|—", "-", text)
    text = re.sub(r"“|”|\"|''", '"', text)
    text = re.sub(r"’|‘|`", "'", text)
    return text

# Load Excel Data
df = pd.read_excel(XLSX_PATH).dropna(subset=["text"])
df["chunk_id"] = df.apply(
    lambda row: f"{row['county'].replace(' ', '')}_{row['report_type'].replace('-', '')}_{row['section'].replace(':', '').replace('.', '').replace(' ', '')}_chunk{row.name}",
    axis=1
)
df["text"] = df["text"].apply(normalize_text)
df["section"] = df["section"].astype(str).apply(normalize_text)


# Use HuggingFace embeddings
embedding_func = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Prepare Documents for LangChain's Chroma.from_documents
documents = [
    Document(
        page_content=row["text"],
        metadata={
            "county": row["county"],
            "report_type": row["report_type"],
            "section": row["section"],
            "page": row["page"],
            "chunk_id": row["chunk_id"] # Include chunk_id in metadata
        }
    )
    for _, row in df.iterrows()
]

# Clear the persistence directory if it exists to avoid conflicts
if os.path.exists(PERSIST_DIR):
    print(f"Clearing existing Chroma directory: {PERSIST_DIR}")
    shutil.rmtree(PERSIST_DIR)


print(f"Creating Chroma collection '{COLLECTION}' at '{PERSIST_DIR}'...")

# Create and populate the Chroma collection
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_func,
    collection_name=COLLECTION,
    persist_directory=PERSIST_DIR
)
vectorstore.persist()

print(f"✅ Chroma collection '{COLLECTION}' created and populated.")

# Optional: Verify document count
print(f"Total documents added to collection: {vectorstore._collection.count()}")

Creating Chroma collection 'sip_csa_chunks' at '/content/drive/MyDrive/[Huggingface Embedding]'...
✅ Chroma collection 'sip_csa_chunks' created and populated.
Total documents added to collection: 15053


/tmp/ipython-input-3887896794.py:68: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [ ]:
# Optional: Run test query
try:
    # result = collection.query(query_texts=["childcare support"], n_results=2)
    # collection has been decreapted by cromadb, I changed to langchain api vectorstore
    results = vectorstore.similarity_search("childcare support", k=2)
    print("\nSample Query Result:")
    for i, r in enumerate(results):
        print(f"Result {i+1}: {r.page_content[:300]}...\n---")
except Exception as e:
    print(f"Error during sample query: {e}")


Sample Query Result:
Result 1: authorization to client (including vouchers, payment, tracking, etc.). Full-time childcare authorization is provided through the Immediate and Continuous Eligibility process. Once a family is referred to WTW, if the family has a child who is under the age of 13, or a child of any age who has a disab...
---
Result 2: in the forms prior to attending their orientation. Upon receipt, CWES Engagement Unit staff approve child care for a 12 month period from the start date listed on the child care packet. Child care can be provided as early as the start date of the client's Cash Aid benefits. 80 For clients already en...
---


In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# Setup
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"
COLLECTION  = "sip_csa_chunks"

embedding_func = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load vectorstore
vectorstore = Chroma(
    collection_name=COLLECTION,
    persist_directory=PERSIST_DIR,
    embedding_function=embedding_func
)

# Create retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Run a query
# use .invoke
docs = retriever.invoke("childcare support")
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---")
    print(f"Metadata: {doc.metadata}")
    print(f"Content: {doc.page_content[:300]}...")





--- Document 1 ---
Metadata: {'section': 'Section 4. Initial Engagement Analysis', 'county': 'San Mateo', 'report_type': 'Cal-CSA', 'chunk_id': 'SanMateo_CalCSA_Section4InitialEngagementAnalysis_chunk804', 'page': 32}
Content: authorization to client (including vouchers, payment, tracking, etc.). Full-time childcare authorization is provided through the Immediate and Continuous Eligibility process. Once a family is referred to WTW, if the family has a child who is under the age of 13, or a child of any age who has a disab...

--- Document 2 ---
Metadata: {'section': 'Section 4. Initial Engagement Analysis', 'county': 'Santa Clara', 'chunk_id': 'SantaClara_CalCSA_Section4InitialEngagementAnalysis_chunk217', 'report_type': 'Cal-CSA', 'page': 72}
Content: in the forms prior to attending their orientation. Upon receipt, CWES Engagement Unit staff approve child care for a 12 month period from the start date listed on the child care packet. Child care can be provided as early as the start d

In [ ]:
import chromadb
from chromadb.config import Settings
PERSIST_DIR = "/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"

client = chromadb.PersistentClient(
    path=PERSIST_DIR,
    settings=Settings(anonymized_telemetry=True)
)


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/CalWorks/Vector Database/Output/chroma_sip_csa_db[Huggingface Embedding]"))  # 看看里面有没有 metadata.json


['chroma.sqlite3', 'af86c337-2244-45e6-9eab-a2770ff68183']


In [ ]:
collections = client.list_collections()

print("📦 Collections and document counts:")
for col in collections:
    # Get the collection object
    c = client.get_collection(col.name)
    print(f"- {col.name}: {c.count()} documents")


📦 Collections and document counts:
- sip_csa_chunks: 15053 documents
